In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'https://chapingo-cori.mx/textual/textual/issue/archive'
url_root = 'https://chapingo-cori.mx/textual/textual/issue/archive'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', class_='container page-archives')
volumen=box.findAll('div', class_='col-md-3 col-lg-2')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://chapingo-cori.mx/textual/textual/issue/view/12',
 'https://chapingo-cori.mx/textual/textual/issue/view/11',
 'https://chapingo-cori.mx/textual/textual/issue/view/10',
 'https://chapingo-cori.mx/textual/textual/issue/view/8',
 'https://chapingo-cori.mx/textual/textual/issue/view/3',
 'https://chapingo-cori.mx/textual/textual/issue/view/1',
 'https://chapingo-cori.mx/textual/textual/issue/view/2',
 'https://chapingo-cori.mx/textual/textual/issue/view/4',
 'https://chapingo-cori.mx/textual/textual/issue/view/5',
 'https://chapingo-cori.mx/textual/textual/issue/view/6',
 'https://chapingo-cori.mx/textual/textual/issue/view/7']

In [10]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='container page-issue') 
    volumen1=box1.findAll('div', class_='article-summary-title')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [11]:
vol2

['https://chapingo-cori.mx/textual/textual/article/view/r.textual.2022.79.01',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2022.79.03',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2022.79.04',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2022.79.05',
 'https://chapingo-cori.mx/textual/textual/article/view/299',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021.77.1',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021.78.02',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021.78.03',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021.78.04',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021.78.05',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021.78.07',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021.78.08',
 'https://chapingo-cori.mx/textual/textual/article/view/r.textual.2021

In [12]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='container page-issue') 
    volumen1=box1.findAll('div', class_='article-summary-title')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [13]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://chapingo-cori.mx/textual/textual/issue/archive


In [14]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

120

In [15]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [16]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Textual'
    a='V. Ciencias Sociales'
    tem='Analisis del Medio Rural'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='article-details-fulltitle').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-details-block article-details-abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='btn btn-primary').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [17]:
list_scraper=list_scraper[0:120]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [18]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Textual,V. Ciencias Sociales,Analisis del Medio Rural,CONFLICTO AMBIENTAL Y TERRITORIALIDADES EN EL ...,Este artículo analiza el conflicto socioambien...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
1,Textual,V. Ciencias Sociales,Analisis del Medio Rural,PARTERÍA TRADICIONAL. VIOLENCIA SISTÉMICA Y FE...,El objetivo de esta contribución es presentar ...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
2,Textual,V. Ciencias Sociales,Analisis del Medio Rural,Patrimonio biocultural en un contexto maya yuc...,Se presenta un análisis de la forma en que el ...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
3,Textual,V. Ciencias Sociales,Analisis del Medio Rural,ESTADO DEL ARTE SOBRE RIESGO SOCIAL Y AMBIENTA...,El presente trabajo tiene la finalidad de eval...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
4,Textual,V. Ciencias Sociales,Analisis del Medio Rural,"¿HIPERINFLACIÓN, UN ARMA PARA DESESTABILIZAR P...",La inflación es un indicador del comportamient...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
...,...,...,...,...,...,...,...,...
115,Textual,V. Ciencias Sociales,Analisis del Medio Rural,Organización y sobrevivencia de los campesinos...,El colapso de la producción de arroz en el est...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
116,Textual,V. Ciencias Sociales,Analisis del Medio Rural,Traspatios campesinos de Morelos,En este artículo se analiza el carácter multif...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
117,Textual,V. Ciencias Sociales,Analisis del Medio Rural,Antropología jurídica y espacio urbano en el V...,En las ciudades habita un número importante de...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...
118,Textual,V. Ciencias Sociales,Analisis del Medio Rural,Programas educativos para fortalecer la sosten...,La generación de programas educativos para for...,None,https://chapingo-cori.mx/textual/textual/artic...,https://chapingo-cori.mx/textual/textual/artic...


In [19]:
df_catalogo.to_csv('Revista179.csv', index=False)